In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
path_archivo = "../input_data/combi_500_100.csv"
path_rand1 = "../input_data/uni_52500_1.csv"
path_rand2 = "../input_data/uni_52500_2.csv"
datos_combi  = pd.read_csv(path_archivo)
datos_rand1 = pd.read_csv(path_rand1)
datos_rand2 = pd.read_csv(path_rand2)

In [3]:
datos_combi = datos_combi.sort_values(by=["eta", "beta", "phi_K"]).reset_index(drop=True)

In [4]:
N_eig = 20
target = "phi_K"
def preprocesar_datos(d_frame, N_eig, target):
    features_eig = list(map(lambda x: "eig_" + str(x+1), range(N_eig)))
    features_dim = ["eta", "beta"]
    feature_especial = ["eig_0"]
    features_tot = [target] + feature_especial + features_dim + features_eig
    dat_copy = d_frame.copy()
    for i in range(N_eig):
        key_mod = "eig_" + str(i+1)
        prev_key = "eig_" + str(i)
        if i == 0:  
            dat_copy[key_mod] = 1/dat_copy[key_mod]
        else:
            dat_copy[key_mod] = dat_copy[prev_key]/dat_copy[key_mod]
        #fin if 
    #fin for
    for i in range(N_eig):
        col = "eig_" + str(i+1)
        dat_copy[col] = np.log(dat_copy[col])
    #fin for 
    dat_copy = dat_copy.dropna()
    problematic_rows = dat_copy[(~np.isfinite(dat_copy)).any(axis=1) | (dat_copy.abs() > np.finfo(np.float64).max).any(axis=1)]
    problematic_indices = problematic_rows.index
    dat_copy = dat_copy.drop(index=problematic_indices)
    return dat_copy[features_tot]
#fin procesar_datos

In [5]:
datos_combi = preprocesar_datos(datos_combi, N_eig, target)
datos_rand1 = preprocesar_datos(datos_rand1, N_eig, target)
datos_rand2 = preprocesar_datos(datos_rand2, N_eig, target)

/home/cubos/.local/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/cubos/.local/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/cubos/.local/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/cubos/.local/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [8]:
metadata_temporal = {"combi": len(datos_combi), "rand1": len(datos_rand1), "rand2": len(datos_rand2)}
print(metadata_temporal)

{'combi': 52500, 'rand1': 52492, 'rand2': 52490}


In [7]:
N_phi_K = 500
N_datos = len(datos_combi)
N_partes = int(N_datos/N_phi_K)
print(N_partes)

105


In [8]:
def graficar_lambdas(data_frame, target, n_eig, preds = {}, op = lambda x: x, nombre = "lineal/", nombre_feat = "lambda_moño"):
    cols_eig = list(map(lambda x: "pca" + str(x), range(n_eig)))
    cols_preds = list(map(lambda x: "pred_" + x, cols_eig)) if len(preds) != 0 else []
    eta = str(np.array(data_frame["eta"])[0]/np.pi)[:4] + "pi"
    beta = str(np.array(data_frame["beta"])[0]/np.pi)[:4] + "pi"
    x = data_frame[target]
    ys = op(data_frame[cols_eig])
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(x, ys)
    if len(preds) != 0:
        ax.plot(preds["y"], ys)
        nombre = "pred/" + nombre
    #fin if 
    ax.set_xlabel(target)
    ax.set_ylabel(nombre_feat)
    ax.set_title("eta=" +  eta + ", beta=" + beta)
    ax.legend(cols_eig + cols_preds)
    plt.savefig("pictures/" + nombre + "eta=" +  eta + ",beta=" + beta + "_" + ".png")
    plt.close()
#fin for 

In [9]:
def graficar_todas(d_frame, n_eig, target, op = lambda x: x, preds = {}, nombre = "lineal/", nombre_feat = "l_moño"):
    for n in range(N_partes):
        start = N_phi_K*n
        end = N_phi_K*(n+1)
        ys = preds["y"][start:end] if len(preds) != 0 else []
        preds_fin = {"y": ys, "X": preds["X"].iloc[start:end]} if len(preds) != 0 else {}
        graficar_lambdas(d_frame.iloc[start:end], target, n_eig, preds_fin, op, nombre, nombre_feat)
    #fin for 
#fin función